<a href="https://colab.research.google.com/github/SurinSeong/FinalPJT/blob/main/langchain_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# langchain-basic
[위키독스-랭체인(LangChain) 입문부터 응용까지](https://wikidocs.net/book/14473)

## 환경 구성

In [ ]:
## 1. 라이브러리 설치
!pip install -q langchain langchain-openai tiktoken # tiktoken : openai에서 사용하는 tokenizer / -q : 설치 중간 과정을 보고 싶지 않을 때 작성

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 581.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


In [ ]:
## 2. openai 인증키 설정
import os

os.environ['OPENAI_API_KEY'] = 'FINAL_TEAM2'

In [ ]:
# Openai를 만들 수 있는 클래스 가져오기
from langchain_openai import ChatOpenAI

# model 생성
llm = ChatOpenAI(model='gpt-3.5-turbo-0125')

# model에게 prompt 전달
llm.invoke('지구의 자전 주기는?') # AIMessage : 모델의 답변

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: FINAL_TEAM2. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# prompt + model + output parser
prompt = ChatPromptTemplate.from_template('You are an expert in astronomy. Answer the question. <Question>: {input}')
llm = ChatOpenAI(model='gpt-3.5-turbo-0125')
output_parser = StrOutputParser() # AIMessage의 content만 반환

# LCEL chaining
chain = prompt | llm | output_parser

# chain 호출
chain.invoke({'input':'지구의 자전 주기는?'})

### 멀티 체인
* 여러 개의 체인을 연결하거나 복합적으로

In [ ]:
# 첫 번째 체인을 위한 prompt
prompt1 = ChatPromptTemplate.from_template('translates {korean_word} to English.')
# 두 번째 체인을 위한 prompt
prompt2 = ChatPromptTemplate.from_template(
    'explain {english_word} using oxford dictionary to me in Korean.'
)

llm = ChatOpenAI(model='gpt-3.5-turbo-0125')

chain1 = prompt1 | llm | StrOutputParser()

chain1.invoke({'korean_word':'미래'})

In [ ]:
chain2 = (
    {'english_word':chain1}
    | prompt2
    | llm
    | StrOutputParser()
)

chain2.invoke({'korean_word':'미래'}) # chain1을 통과하는 것이 먼저 이기 때문에 korena_word를 넣어야 한다.

### PromptTemplate
* 단일문장입력 --> 단일 문장 출력

In [ ]:
from langchain_core.prompts import PromptTemplate

# 'name'과 'age'라는 두 개의 변수를 사용하는 프롬프트 템플릿을 정의한다.
template_text = '안녕하세요. 제 이름은 {name}이고, 나이는 {age}살 입니다.'

# PromptTemplate 인스턴스를 생성한다.
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성한다.
filled_prompt = prompt_template.format(name='홍길동', age=30)

filled_prompt

In [ ]:
# 문자열 템플릿 결합 (+ 연산자 이용 가능)
combined_prompt = (
    prompt_template
    + PromptTemplate.from_template('\n\n아버지를 아버지라 부를 수 없습니다.')
    + '\n\n{language}로 번역해주세요.'
)

combined_prompt

In [ ]:
combined_prompt.format(name='홍길동', age=30, language='영어')

### ChatPromptTemplate
* message 여러 개가 리스트 형태로 배열을 이룬다.
* 여러 의미를 가진 메세지를 받는다.
* 챗봇형태의 모델을 구현할 때 자주 사용함.

In [ ]:
## 메시지 구성 방법 1. 튜플 형태의 메시지 목록으로 프롬르트 생성 (role, content)
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate..from_messages([
    ('system' : '이 시스템은 천문학 질문에 답변할 수 있습니다.'),
    ('user' : '{user_input}'), # user_input : 입력받아야할 메시지
])

messages = chat_prompt.format_messages(user_input='태양계에서 가장 큰 행성은 무엇인가요?')
# format_messages : 메시지들로부터 prompt를 만드는 메소드
messages # SystemMessage , HumanMessage 두 가지의 message가 출력된다.

In [ ]:
chain = chat_prompt | llm | StrOutputParser()

chain.invoke({'user_input' : '태양계에서 가장 큰 행성은 무엇인가요?'})

In [ ]:
## 메시지 구성 방법 2. MessagePromptTemplate 활용해서 프롬프트 생성하기
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template('이 시스템은 천문학 질문에 답변할 수 있습니다.'),
        HumanMessagePromptTemplate.from_template('{user_input}'),
    ]
)

messages = chat_prompt.format_messages(user_input='태양계에서 가장 큰 행성은 무엇인가요?')
messages

In [ ]:
chain = chat_prompt | llm | StrOutputParser()
chain.invoke({'user_input' : '태양계에서 가장 큰 행성은 무엇인가요?'})

## Model Parameter
1. Langchain 모델 유형
    1. LLM
    2. Chat Model

In [ ]:
## LLM ##
from langchain_openai import OpenAI

llm = OpenAI()

llm.invoke('한국의 대표적인 관광지 3군데를 추천해주세요.')

In [ ]:
## Chat Model ## 채팅에 특화되어 있음.
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chat = ChatOpenAI()

chat_prompt = ChatPromptTemplate.from_messages([
    ('system', '이 시스템은 여행 전문가 입니다.'),
    ('user', '{user_input}'),
]) # 두 개의 메시지가 전달된다. 메시지 유형은 정해져 있음.

chain = chat_prompt | chat
chain.invoke({'user_input':'안녕하세요? 한국의 대표적인 관광지 3군데를 추천해주세요.'})

* Temperature : 생성된 텍스트의 다양성 조정
    * 값이 작으면 예측가능하고 일관된 출력 생성, 값이 크면 다양하고 예측하기 어려운 출력 생성
* Max Token : 생성할 최대 토큰 수 지정. 생성할 텍스트의 길이를 제한한다.
* Top P : 생성과정에서 특정 확률 분포 내에서 상위 P% 토큰만을 고려하는 방식
* Frequency Penalty : 값이 클수록 이미 등장한 단어나 구절이 다시 등장할 확률을 감소시킨다. 반복을 줄이고 텍스트의 다양성 증가
* Presence Penalty : 텍스트 내에서 단어의 존재 유무에 따라 그 단어의 선택 확률을 조정한다.
* Stop Sequence : 특정 단어나 구절이 등장할 경우 생성을 멈추도록 설정

In [ ]:
## 모델 생성 시점 : 모델에 직접 파라미터 전달
from langchain_openai import ChatOpenAI

# 모델 파라미터 설정
params = {
    'temperature':0,
    'max_tokens':100,
}

kwargs = {
    'frequency_penalty':0.5,
    'presence_penalty':0.5,
    'stop':['\n']
}

# 모델 인스턴스를 생성할 때 설정
model = ChatOpenAI(model='qpt-3.5-turbo-0125', **params, model_kwargs=kwargs)

# 모델 호출
question = '태양계에서 가장 큰 행성은 무엇인가요?'
response = model.invoke(input=question)

# 전체 응답 출력
print(response)

In [ ]:
## 모델 호출 시점 : 모델에 직접 파라미터 전달
# 모델 파라미터 설정
params = {
    'temperature':0.7,
    'max_tokens':10,
}

# 모델 인스턴스를 호출할 때 전달
response = model.invoke(input=question, **params)

# 문자열 출력
print(response.content)

모델에 추가적인 파라미터 전달

* bind 메서드를 사용해서 체인에 새로운 파라미터를 추가해서 연결 가능
* 다양한 상황에 맞게 모델의 동작을 제어

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    {'system':'이 시스템은 천문학 질문에 답변할 수 있습니다.'},
    {'user':'{user_input}'},
])

model = ChatOpenAI(model='gpt-3.5-turbo-0125', max_tokens=100) # 처음 모델을 설정할 때 max_tokens 설정

messages = prompt.format_messages(user_input='태양계에서 가장 큰 행성은 무엇인가요?')

before_answer = model.invoke(messages)

## binding 이전 출력
print(before_answer)

# 모델 호출 시, 추가적인 인수를 전달하기 위해 bind 메서드 사용
chain = prompt | model.bind(max_tokens=10) # 모델의 max_tokens 수 변경

after_answer = chain.invoke(messages)

# binding 이후 출력
print(after_answer)

## RAG
1. 기본 구조
* 검색 단계 : 사용자의 질문이나 컨텍스트를 입력으로 받아, 이와 관련된 외부 데이터를 검색하는 단계. 검색 엔진(API)이나 데이터 베이스(vector store) 등 다양한 소스에서 필요한 정보를 찾아낸다. 검색된 데이터는 질문에 대한 답변을 생성하는데 적합하고 상세한 정보를 포함하는 것을 목표로 한다.
* 생성 단계 : 검색된 데이터를 기반으로 LLM 모델이 사용자의 질문에 답변을 생성하는 단계. 이 단계에서 모델은 검색된 정보와 기존의 지식을 결합해, 주어진 질문에 대한 답변을 생성

2. 과정
* 사용될 질문이 주어졌을 때, vector store에 있는 문서들(embedding 된 형태로 텍스트들을 청크 단위로 작은 조각으로 나눠줌) 중 가장 관련이 깊은 문서를 찾는다.
* 문서를 찾아서 프롬프트를 만들 때 질문과 함께 검색해 놓은 chunk를 결합해서 새로운 prompt를 만들고 새로운 prompt를 llm이 받는다.

In [2]:
# 패키지 설치
!pip install -q langchain langchain-openai tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00

In [4]:
!pip install -q langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.1 MB/s eta 0:00:00


In [3]:
# langchain 버전 확인하기
import langchain

langchain.__version__

'0.2.12'

In [ ]:
## openAI 인증키 설정
import os

os.environ['OPENAI_API_KEY'] = 'FINAL_TEAM2'

## RAG 파이프라인 개요
* load data -text split - indexing - retrieval - generation

In [ ]:
## step 1. Load DATA

# data_loader - 웹페이지 데이터 가져오기
from langchain_community.document_loaders import WebBaseLoader

# 위키피디아 정책과 지침
url = 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8'
loader = WebBaseLoader(url)

# 웹페이지 텍스트 --> documents
docs = loader.load()

print(len(docs))
print(len(docs[0].page_content))
print(docs[0].page_content[5000:6000])

In [ ]:
## step 2. split texts : 하나의 긴 문서를 작은 조각으로 나누기
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# 나눠진 조각의 개수 확인
print(len(splits))
print(splits[10])

In [ ]:
# page_content 속성
splits[10].page_content

In [ ]:
# metadata 속성
splits[10].metadata

In [ ]:
## step 3. indexing
# indexing (Texts --> Embedding (chunk를 벡터 임베딩으로 바꿔주는 작업 필요) --> Store (임베딩으로 변환된 벡터를 벡터 스토어에 저장))
from langchain_community.vectorstores import Chroma # 벡터 스토어
from langchain_openai import OpenAIEmbeddings # 임베딩 모델

vectorstore = Chroma.from_documents(documents=splits, # 임베딩할 문서
                                    embedding=OpenAIEmbeddings()) # 어떤 임베딩 모델을 적용할 것인가?

docs = vectorstore.similarity_search('격하 과정에 대해서 설명해주세요.') # 유사도 기반의 검색

print(len(docs))
print(docs[0].page_content)

In [ ]:
## step 4. retrieval ~ generation
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Prompt
# 일반적으로 RAG에서 많이 사용하는 template
template = \
'''
Answer the question based only on the following context:
[context]

Question: [question]
'''

prompt = ChatPromptTemplate.from_template(template)

# LLM
model = ChatOpenAI(model='gpt-3.5-turbo-0125', temperature=0)

# Retriever (vectorstore에서 검색해줘)
retriever = vectorstore.as_retriever()

# Combine Documents
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG chain 연결
rag_chain = (
    {'context':retriever | format_docs, 'question':RunnablePassthrough()} # 검색기가 가져온 여러 개의 문서를 format_docs를 이용해 하나의 문서로 합쳐준다. / 'question' : RunnablePassthrough를 통해 사용자가 키보드로 입력한 것이 질문으로 전달이 되도록 설정
    | prompt # 위에서 받은 context, question을 prompt에 전달한다.
    | model
    | StrOutputParser()
)

# Chain 실행
rag_chain.invoke('격하 과정에 대해서 설명해주세요.')